<a href="https://colab.research.google.com/github/kantasis/SalesAnalysis/blob/main/Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git status

!ls -l drive/MyDrive/techpro/final
# total 4
# drwxr-xr-x 1 root root 4096 Jan 19 14:20 sample_data

# !pwd

total 45
-rw------- 1 root root 45541 Jan 23 15:57 adjusted_retail_sales_data_v2.csv


In [ ]:
# Parameters:
mountpoint_rpath = '/content/drive'
dataset_rfile = f'{mountpoint_rpath}/MyDrive/techpro/final/adjusted_retail_sales_data_v2.csv'

# Imports:
from google.colab import drive
drive.mount(mountpoint_rpath)

import pandas as pd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rawData_df = pd.read_csv(dataset_rfile)
rawData_df.describe()



,CustomerAge
count,1000.000000
mean,29.177000
std,11.680844
min,2.000000
25%,22.000000
50%,22.000000
75%,35.000000
max,200.000000


In [ ]:
rawData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   SalesDate         1000 non-null   object
 1   ProductCategory   1000 non-null   object
 2   SalesAmount       1000 non-null   object
 3   CustomerAge       1000 non-null   int64 
 4   CustomerGender    1000 non-null   object
 5   CustomerLocation  998 non-null    object
 6   ProductRatings    1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [ ]:
rawData_df.head()

,SalesDate,ProductCategory,SalesAmount,CustomerAge,CustomerGender,CustomerLocation,ProductRatings
0,2023-10-15,Home Appliances,609,22,Male,USA,4
1,2023-09-16,Clothing,1367,22,Female,Australia,5
2,2022-09-06,Electronics,1736,22,Male,UK,2
3,2023-02-24,Female,1838,35,Clothing,India,2
4,2022-09-24,Home Appliances,1829,35,Male,UK,5
